### Package installation

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


### Package call

In [22]:
import psycopg2
import pandas as pd
import psycopg2.extras

### Extract

***Read from excel file***

In [8]:
DATA_DIR = "C:/Users/User/vintern/data/"

In [9]:
trans_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=1, engine='openpyxl')

In [37]:
import pandas as pd

coustomer_dim = pd.read_csv(DATA_DIR + 'Coustomer_dim.csv')

print(coustomer_dim[:200]) 

     coustomer_key             name    contact_no           nid
0          C000001            sumit  8.801920e+12  7.505080e+12
1          C000002         tammanne  8.801817e+12  1.977730e+12
2          C000003    kailash kumar  8.801660e+12  3.769490e+12
3          C000004  bhagwati prasad  8.801534e+12  9.378830e+12
4          C000005             ajay  8.801944e+12  3.540820e+12
...            ...              ...           ...           ...
9186       C009187        smt radha  8.801560e+12  1.057820e+12
9187       C009188     tulsi sharma  8.801790e+12  7.970880e+12
9188       C009189        madhubala  8.801890e+12  3.101930e+12
9189       C009190       pintu devi  8.801560e+12  7.234330e+12
9190       C009191            roopa  8.801520e+12  5.682730e+12

[9191 rows x 4 columns]


In [10]:
trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [49]:
print(coustomer_dim.values) 
print(coustomer_dim) 

[['C000001' 'sumit' 8801920000000.0 7505080000000.0]
 ['C000002' 'tammanne' 8801817069329.0 1977730000000.0]
 ['C000003' 'kailash kumar' 8801660000000.0 3769490000000.0]
 ...
 ['C009189' 'madhubala' 8801890000000.0 3101930000000.0]
 ['C009190' 'pintu devi' 8801560000000.0 7234330000000.0]
 ['C009191' 'roopa' 8801520000000.0 5682730000000.0]]
     coustomer_key             name    contact_no           nid
0          C000001            sumit  8.801920e+12  7.505080e+12
1          C000002         tammanne  8.801817e+12  1.977730e+12
2          C000003    kailash kumar  8.801660e+12  3.769490e+12
3          C000004  bhagwati prasad  8.801534e+12  9.378830e+12
4          C000005             ajay  8.801944e+12  3.540820e+12
...            ...              ...           ...           ...
9186       C009187        smt radha  8.801560e+12  1.057820e+12
9187       C009188     tulsi sharma  8.801790e+12  7.970880e+12
9188       C009189        madhubala  8.801890e+12  3.101930e+12
9189       C0091

### Transform

### Load

***Connect to the database***

In [24]:

class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "15556010",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [25]:
con = PostgresConnection().getConnection()

successfully connected to database


In [13]:
insert_stmt = "INSERT INTO ecomdb_star_schema.trans_dim ( payment_key, trans_type, bank_name)" \
            " VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, trans_dim.values)
con.commit()
cur.close()

In [48]:
insert_cus = "INSERT INTO ecomdb_star_schema.coustomer_dim ( coustomer_key, name, contact_no, nid)" \
            " VALUES (%s, %s, %s , %s)"
cur2 = con.cursor()
psycopg2.extras.execute_batch(cur2, insert_cus, coustomer_dim[:10].values)
con.commit()
cur2.close()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [14]:
cur = con.cursor()
select_stmt = "SELECT t.payment_key , t.trans_type, t.bank_name " \
              "FROM ecomdb_star_schema.trans_dim t"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('P001', 'cash', 'None'),
 ('P002', 'card', 'AB Bank Limited'),
 ('P003', 'card', 'Bangladesh Commerce Bank Limited'),
 ('P004', 'card', 'Bank Asia Limited'),
 ('P005', 'card', 'BRAC Bank Limited'),
 ('P006', 'card', 'Citizens Bank Limited'),
 ('P007', 'card', 'City Bank Limited'),
 ('P008', 'card', 'Community Bank Bangladesh Limited'),
 ('P009', 'card', 'Dhaka Bank Limited'),
 ('P010', 'card', 'Dutch-Bangla Bank Limited'),
 ('P011', 'card', 'Eastern Bank Limited'),
 ('P012', 'card', 'IFIC Bank Limited'),
 ('P013', 'card', 'Jamuna Bank Limited'),
 ('P014', 'card', 'Meghna Bank Limited'),
 ('P015', 'card', 'Mercantile Bank Limited'),
 ('P016', 'card', 'Midland Bank Limited'),
 ('P017', 'card', 'Modhumoti Bank Limited'),
 ('P018', 'card', 'Mutual Trust Bank Limited'),
 ('P019', 'card', 'National Bank Limited'),
 ('P020', 'card', 'National Credit & Commerce Bank Limited'),
 ('P021', 'card', 'NRB Bank Limited'),
 ('P022', 'card', 'NRB Commercial Bank Ltd'),
 ('P023', 'card', 'NRB Global B

In [15]:
trans_df = pd.DataFrame(list(records), columns=['payment_key', 'trans_type', 'bank_name'])
trans_df

,payment_key,trans_type,bank_name
0,P001,cash,None
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited
